In [1]:
library(rwwa)

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: plyr


Attaching package: ‘plyr’


The following object is masked from ‘package:distillery’:

    is.formula




# Per grid cell

In [2]:
gmst <- read.table("gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2023]

nino <- read.csv("nino34_djf.csv", col.names = c("year", "nino"))

cov_2023 <- data.frame(gmst = gmst$gmst[gmst$year == 2023], nino = nino$nino[nino$year == 2023])
cov_cf <- rbind("hist" = cov_2023 - c(1.2, 0),
                "neut" = c(cov_2023$gmst, 0))

In [2]:
fl = list.files(pattern = "tamsat_pr-djf_flattened")

for(fnm in fl) {
    
    res_fnm <- paste0("gridded-res_",gsub("_flattened", "", fnm))
    
    if (!file.exists(res_fnm)) {
        df <- read.csv(fnm, skip = 3, header = F)
        colnames(df)[1] <- "year"
        df <- merge(merge(nino, gmst), df)

        # fit nonstationary model to all grid cells
        models <- sapply(colnames(df)[-(1:3)], function(cnm) {
            df_c <- df[,c("year", "gmst", "nino", cnm)]
            tryCatch({
                fit_ns("norm", "fixeddisp", df_c, cnm, c("gmst", "nino"), lower = T)
            }, error = function(cond) { return(NULL) })
        }, simplify = F)

        # get all model results & save as .csv to be reconstructed & plotted in python
        fitted_res <- sapply(models, function(mdl) {
            if(!is.null(unlist(mdl)[1])) {
                c(mdl_ests(mdl, cov_f = cov_2023, cov_cf = cov_cf), "rp_neut" = return_period(mdl, x = mdl$ev, fixed_cov = cov_cf["neut",,drop=F]))
            } else {
                rep(NA, 14)
            }})
        write.csv(fitted_res, res_fnm)
    }
}

# Unconditional return periods

In [33]:
gmst <- read.table("gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2024]
nino <- read.csv("nino34_djf.csv", col.names = c("year", "nino"))
pr <- read.csv("era5_prDJF.dat", sep = " ", col.names = c("year", "pr"))

df <- merge(merge(gmst, nino), pr)

mdl <- fit_ns("norm", "fixeddisp", df, "pr", c("gmst", "nino"), lower = T)

In [34]:
# return period in 2024 climate, given the observed distribution of Nino events
1/(mean(sapply(df$nino, function(i) {
        1/return_period(mdl, mdl$ev, fixed_cov = data.frame("gmst" = 0, "nino" = i))
})))

[1] 8.208197

In [32]:
read.csv("mswep_prDJF.dat", sep = " ", col.names = c("year", "pr"))

year,pr
<int>,<dbl>
1979,NA
1980,329.36267
1981,239.77788
1982,206.89740
1983,232.29886
1984,276.96112
1985,397.43973
1986,366.64304
1987,250.10803
